In [1]:
import pandas as pd
import uproot
#import uproot3
import numpy as np
# import track_time_calibration as ttc
from matplotlib import pyplot as plt
import math
import sys
import os 
import fnmatch
sys.path.insert(1, r'C:\Users\eliot\EPFL\TP4_ECAL\Code\ecal_reco\utils')
sys.path.insert(1, r'C:\Users\eliot\EPFL\TP4_ECAL\Code\ecal_reco\tracking')
sys.path.insert(1, r'C:\Users\eliot\EPFL\TP4_ECAL\Code\ecal_reco\time_alignement')
sys.path.insert(1, r'C:\Users\eliot\EPFL\TP4_ECAL\Code\ecal_reco\muon_decay')
from track import Track
from track3D import Track3D
from hit import Hit
from data_loading import *
from parameters import *
from tqdm import tqdm
from track_reconstruction import *
from physics import dist_line_rect
import pickle
from IPython import display
from find_muon_decay import find_muon_decay
from time_correction import *
import copy
import nbformat

selected 75.80% of all events


In [2]:
## test of electronics PCB time correction function

Hx = Hit([1,1],1,0.4,22630,0.1)
Tx = Track([Hx])
Hy = Hit([3,2],0,0.45,22630,0.1)
Ty = Track([Hy])
T = Track3D(Tx,Ty)
print("original timestamp = " + str(T.x.hits[0].timestamp))

h = copy.deepcopy(T.x.hits[0])
[x,z] = h.coord
tofpet = mapping_inv_2D(1,x,z)
h.timestamp = h.timestamp - mapping_SiPM_delay(tofpet[0], tofpet[1])

print("delay = " + str(mapping_SiPM_delay(tofpet[0], tofpet[1])))
#T = time_correction_electronics(T)
print("timestamp after correction = " + str(h.timestamp))
print("addition of new timestamp and delay = " + str(h.timestamp+mapping_SiPM_delay(tofpet[0], tofpet[1])))


print("orignal timestamp not modified : " + str(T.x.hits[0].timestamp))

original timestamp = 0.4
delay = 0.24155552
timestamp after correction = 0.15844448000000003
addition of new timestamp and delay = 0.4
orignal timestamp not modified : 0.4


In [3]:
## Test of the deepcopy 

print("original timestamp = " + str(T.x.hits[0].timestamp))

T2 = copy.deepcopy(T)
T2 = time_correction_electronics(T2)
print("copied timestamp = " + str(T2.x.hits[0].timestamp))


print("orignal timestamp not modified : " + str(T.x.hits[0].timestamp))

original timestamp = 0.4
copied timestamp = 0.15844448000000003
orignal timestamp not modified : 0.4


In [4]:
## test of the time correction fiber function

x =1
Hx1 = Hit([x,8],1,0,1000,0.1) #(coord, is_sidex, timestamp, timestamp_event, value)
Hx2 = Hit([x+1,7],1,0,1000,0.1)
Hx3 = Hit([x+3,6],1,0,1000,0.1)
Tx = Track([Hx1,Hx2,Hx3])
y = 1
Hy1 = Hit([y,8],0,0,1000,0.1)
Hy2 = Hit([y+2,7],0,0,1000,0.1)
Hy3 = Hit([y+4,6],0,0,1000,0.1)
Ty = Track([Hy1,Hy2,Hy3])
T = Track3D(Tx,Ty)
T.show()


T=time_correction_fiber(T)
print("timestamp x1 after correction = " + str(T.x.hits[0].timestamp))
print("timestamp x2 after correction = " + str(T.x.hits[1].timestamp))
print("timestamp y after correction = " + str(T.y.hits[0].timestamp))



timestamp x1 after correction = -0.041213930382045204
timestamp x2 after correction = -0.09524919855354277
timestamp y after correction = -0.0011407407407406254


In [5]:
## Test of all the time correction functions, with simulated tracks

x = 21
Hx1 = Hit([x,8],1,0,1000,0.1) #(coord, is_sidex, timestamp, timestamp_event, value)
Hx2 = Hit([x+1,7],1,0,1000,0.1)
Hx3 = Hit([x+3,6],1,0,1000,0.1)
Tx = Track([Hx1,Hx2,Hx3])
y = 1
Hy1 = Hit([y,8],0,0,1000,0.1)
Hy2 = Hit([y+2,7],0,0,1000,0.1)
Hy3 = Hit([y+4,6],0,0,1000,0.1)
Ty = Track([Hy1,Hy2,Hy3])
T = Track3D(Tx,Ty)
T.show()

# T=time_correction_electronics(T)
T=time_correction_fiber(T)
# T=time_correction_offset(T)
# T= time_correction_global(T)

Xtimestamp =[]
Ytimestamp = []
for h in T.x.hits :
    Xtimestamp.append(h.timestamp)
for h in T.y.hits :
    Ytimestamp.append(h.timestamp)

print(Xtimestamp)
print(Ytimestamp)

[-0.041213930382045204, -0.09524919855354277, -0.1492844667250404]
[-0.342474074074074, -0.3876945274307568, -0.43291498078743956]


In [7]:
## Test of find_muon_decay function with time correction (specificly mean_timestamp())

x = 21
Hx1 = Hit([x,8],1,25,1000,0.1) #(coord, is_sidex, timestamp, timestamp_event, value)
Hx2 = Hit([x+1,7],1,20,1000,0.1)
Hx3 = Hit([x+3,6],1,30,1000,0.1)
Tx = Track([Hx1,Hx2,Hx3])
y = 1
Hy1 = Hit([y,8],0,15,1000,0.1)
Hy2 = Hit([y+2,7],0,15,1000,0.1)
Hy3 = Hit([y+4,6],0,10,1000,0.1)
Ty = Track([Hy1,Hy2,Hy3])
T = Track3D(Tx,Ty)

print(mean_timestamp([Hx1,Hx2,Hx3]))

AttributeError: 'list' object has no attribute 'timestamp'